Import

In [145]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
sklearn.__version__

'1.7.1'

### Trick 01 - کاربرد Random State در Scikit-Learn


In [146]:
df = pd.read_csv("../../00 DATA/titanic.csv")
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [147]:
cols = ['fare','embarked','sex']
x = df[cols]
y = df['survived']

In [148]:
from sklearn.model_selection import train_test_split

In [149]:
x

,fare,embarked,sex
0,211.3375,S,female
1,151.5500,S,male
2,151.5500,S,female
3,151.5500,S,male
4,151.5500,S,female
...,...,...,...
1304,14.4542,C,female
1305,14.4542,C,female
1306,7.2250,C,male
1307,7.2250,C,male


In [150]:
x_test,x_train,y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=1)
x_train.head()

,fare,embarked,sex
201,51.8625,S,male
115,263.0000,S,male
255,69.3000,C,female
1103,39.6875,S,male
195,86.5000,S,female


In [151]:
x_test,x_train,y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=1)
x_train.head()

,fare,embarked,sex
201,51.8625,S,male
115,263.0000,S,male
255,69.3000,C,female
1103,39.6875,S,male
195,86.5000,S,female


In [152]:
# هر بار که این دستور رو اجرا کنیم باز هم یک دیتا رو میگیرم چون رندوم استیت یک هست


In [153]:
x_test,x_train,y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=42)
x_train.head()

,fare,embarked,sex
1148,7.1250,S,male
1049,15.7417,C,male
982,7.8958,S,male
808,8.0500,S,male
1195,7.7500,Q,male


In [154]:
x_test,x_train,y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=42)
x_train.head()

,fare,embarked,sex
1148,7.1250,S,male
1049,15.7417,C,male
982,7.8958,S,male
808,8.0500,S,male
1195,7.7500,Q,male


In [155]:
# اما وقتی رندوم استیت رو عوض کنیم هر بار تفاوت خواهیم داشت

In [156]:
del df,x,y,x_test,x_train,y_train,y_test, cols
gc.collect()

12

### Trick 02 - کاربرد Stratify در Scikit-Learn

In [157]:
# هدف اینه که تقسیم بندی دیتا تست و ترین رو به یک نوع انجام دهیم که که تناسب کفیت داشته باشن


df = pd.DataFrame({'feature':list(range(12)), 'target':['Not Fraud']*10 + ['Fraud']*2})
#dft02.sample()


In [158]:
x = df['feature']
y = df['target']


In [159]:
from sklearn.model_selection import train_test_split

#### **Not stratified**

`y_train` contains **NONE** of the minority class, whereas `y_test` contains **ALL** of the minority class. (This is bad!)

In [160]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=0)

In [161]:
y_train

1    Not Fraud
7    Not Fraud
9    Not Fraud
3    Not Fraud
0    Not Fraud
5    Not Fraud
Name: target, dtype: object

In [162]:
y_test

6     Not Fraud
11        Fraud
4     Not Fraud
10        Fraud
2     Not Fraud
8     Not Fraud
Name: target, dtype: object

#### Stratified

Class proportions are the **SAME** in `y_train` and `y_test`. (This is good!)

In [163]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=0, stratify=y)

In [164]:
y_train

2     Not Fraud
8     Not Fraud
4     Not Fraud
1     Not Fraud
11        Fraud
9     Not Fraud
Name: target, dtype: object

In [165]:
y_test

0     Not Fraud
7     Not Fraud
3     Not Fraud
5     Not Fraud
10        Fraud
6     Not Fraud
Name: target, dtype: object

#### Delete

In [166]:
del df,x,y,x_test,x_train,y_train,y_test
gc.collect()

12

### Trick 03 - کاربرد Missing Indicator در Scikit-Learn

In [167]:
#این مثال رو در نظر بگیرید

x = pd.DataFrame({
    'age':[20,30, 10 , np.nan, 10]
})
x

,age
0,20.0
1,30.0
2,10.0
3,NaN
4,10.0


In [169]:
""" ما نمی تونیم دیتایی که نات نامبر هست رو نگر داریم برای همین باید یه فکری براش بکنیم
مثلا باید دراپشون کنیم
یکی از راها اینه که با میانگین گرفتن از بقیه ریکورد ها جاشو پر کنیم
"""
from sklearn.impute import SimpleImputer

In [184]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#'constant', 'median', 'most_frequent', 'mean'
# imputer = SimpleImputer()
imputer.fit_transform(x)


array([[20. ],
       [30. ],
       [10. ],
       [17.5],
       [10. ]])

In [183]:
imputer = SimpleImputer(missing_values=np.nan, strategy = 'median')
#'constant', 'median', 'most_frequent', 'mean'
# imputer = SimpleImputer()
imputer.fit_transform(x)


array([[20.],
       [30.],
       [10.],
       [15.],
       [10.]])

In [180]:
# پرچم گذاری : برای اینکه بفهمیم این داده میس ولو هست و توسط ما پر شده
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = SimpleImputer(add_indicator=True)
imputer.fit_transform(x)


array([[20. ,  0. ],
       [30. ,  0. ],
       [10. ,  0. ],
       [17.5,  1. ],
       [10. ,  0. ]])